<a href="https://colab.research.google.com/github/teku-teku/docker-django-postgresql/blob/master/create_interview_sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import csv
import openpyxl

import os
os.chdir('/content/drive/MyDrive/選考業務自動化プログラム/申込者データ・面接シート作成/')

def main():
  applicant_list = []
  with open('申込者データ/申込者一覧.csv', 'r',encoding="utf-8") as f:
    for row in csv.DictReader(f):
      if row["氏名"]:
        applicant_list.append(row)

  form_list = []
  with open('申込者データ/forms.csv', 'r',encoding="utf-8") as f:
    for row in csv.DictReader(f):
      form_list.append(row)

  result_list = []
  sort_interview_time = []


  if form_list:
    #JICOOのデフォルト項目の名寄せ
    if "name" in form_list[0]:
      form_name_column = "name"
    elif "名前" in form_list[0]:
      form_name_column = "名前"
    else:
      print("[ERROR]JICOOのデータに「名前（name）」の項目が含まれていません")
      return

    if "email" in form_list[0]:
      form_email_column = "email"
    elif "メールアドレス" in form_list[0]:
      form_email_column = "メールアドレス"
    else:
      print("[ERROR]JICOOのデータに「メールアドレス（email）」の項目が含まれていません")
      return

    if "reservation start time" in form_list[0]:
      form_reservation_column = "reservation start time"
    elif "予約開始時間" in form_list[0]:
      form_reservation_column = "予約開始時間"
    else:
      print("[ERROR]JICOOのデータに「予約開始時間（reservation start time）」の項目が含まれていません")
      return

    #changed item name
    if "ハローワーク住所（記入例：184-0000\u3000東京都新宿区新宿1-0-0）※郵便番号は必須" in form_list[0]:#2月8日開講の講座までこのヘッダー
      form_address_column = "ハローワーク住所（記入例：184-0000\u3000東京都新宿区新宿1-0-0）※郵便番号は必須"
    elif "ハローワーク住所（記入例：東京都新宿区新宿1-0-0）※受講申込書通りにご記入ください" in form_list[0]:
      form_address_column = "ハローワーク住所（記入例：東京都新宿区新宿1-0-0）※受講申込書通りにご記入ください"
    else:
      print("[ERROR]JICOOのデータに「ハローワーク住所」の項目が含まれていないか、変更がありました。開発者に伝えてください")
      return

  for i in range(len(applicant_list)):
    for form_item in form_list:
      if (applicant_list[i]["電話番号\n(携帯優先)"] and applicant_list[i]["電話番号\n(携帯優先)"].strip() == form_item['電話番号（090-0000-0000）'].strip()) \
        or (applicant_list[i]["メールアドレス"] and applicant_list[i]["メールアドレス"].strip() == form_item[form_email_column].strip()) \
        or (applicant_list[i]["氏名"] and applicant_list[i]["氏名"].replace(" ", "").replace("　", "") == form_item[form_name_column].replace(" ", "").replace("　", "")):
        applicant_list[i]["誕生日"] = form_item['生年月日（記入方法：1999/1/1\u3000半角数字）'] if not applicant_list[i]["誕生日"] else applicant_list[i]["誕生日"]
        applicant_list[i]["よみがな"] = form_item['ふりがな'] if not applicant_list[i]["よみがな"] else applicant_list[i]["よみがな"]
        applicant_list[i]["ハローワーク名"] = form_item['受付ハローワーク名（記入方法：渋谷公共職業安定所）※受講申込書通りにご記入ください'] if not applicant_list[i]["ハローワーク名"] else applicant_list[i]["ハローワーク名"]
        applicant_list[i]["ハローワーク住所"] = form_item[form_address_column] if not applicant_list[i]["ハローワーク住所"] else applicant_list[i]["ハローワーク住所"]
        applicant_list[i]["電話番号"] = form_item['ハローワーク電話番号（03-0000-0000）※受講申込書通りにご記入ください'] if not applicant_list[i]["電話番号"] else applicant_list[i]["電話番号"]
        applicant_list[i]["電話番号\n(携帯優先)"] = form_item['電話番号（090-0000-0000）'] if not applicant_list[i]["電話番号\n(携帯優先)"] else applicant_list[i]["電話番号\n(携帯優先)"]
        applicant_list[i]["面接時間"] = form_item[form_reservation_column].split(" ")[1][:-3]
        #追加の項目
        if "ハローワーク郵便番号（記入例：184-0000）※郵便番号のみを記載" in form_item:
          applicant_list[i]["郵便番号"] = form_item["ハローワーク郵便番号（記入例：184-0000）※郵便番号のみを記載"]
        if (form_item[form_reservation_column]):
          sort_interview_time.append(form_item[form_reservation_column].split(" ")[1][:-3])
        break
    result_list.append(applicant_list[i])

  with open('申込者データ/結合済_申込データ.csv', 'w',encoding="utf-8") as f:
    header = result_list[0].keys()
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    writer.writerows(result_list)

  #create interview sheet
  wb = openpyxl.load_workbook('面接シート/面接シート_テンプレート.xlsx')

  if not '別紙２(記録)' in wb.sheetnames:
    print("[ERROR]シート「別紙２(記録)」が含まれていません")
    return

  sheet = wb['別紙２(記録)']

  sort_interview_time.sort()
  for sort_flag in sort_interview_time:
    for i in range(len(result_list)):
      if result_list[i]['面接時間'] and result_list[i]['面接時間'] == sort_flag:
        sheet_copy = wb.copy_worksheet(wb['別紙２(記録)'])
        interview_time = result_list[i]['面接時間'].replace(":", "")
        if len(interview_time) == 3:
            interview_time = "0" + interview_time
        name = result_list[i]['氏名']
        sheet_copy.title = interview_time + name
  #copy file
  wb.save(f'面接シート/面接シート.xlsx')

main()

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
